In [1]:
!python -m venv C:\GreenGIS\NearMissDash
!C:\GreenGIS\NearMissDash\Scripts\activate.bat
!PS C:\GreenGIS\NearMissDash\Scripts\Activate.ps1

      PID    PPID    PGID     WINPID   TTY         UID    STIME COMMAND
    69980       1   69980      69980  ?         197610 08:55:22 /usr/bin/ps


In [20]:
#import packages
from flask import Flask, render_template, request, jsonify 
from flask_json import FlaskJSON, JsonError, json_response
from flask_mail import Mail, Message 
import folium
from IPython.display import HTML, display
import json
import pandas as pd
import numpy as np
import os
from collections import Counter
import io
import smtplib
from email.mime.text import MIMEText
from waitress import serve
import requests
from bokeh.embed import file_html 
from bokeh.plotting import figure, show 
from bokeh.resources import CDN

#name app
app = Flask(__name__)
app.config['SECRET_KEY'] = 'greenstreets'

#setup JSONBin.io 
JSONBIN_ACCESS_KEY = '$2a$10$7B8tVje037JkjFrFjBuXauoSw5x7XdSVhSL2KfUqm7XfcNSOhQ8ae'
JSONBIN_API_URL = f'https://api.jsonbin.io/v3/b/66ce2fe9e41b4d34e42626f8'
BIN_ID = '66ce2fe9e41b4d34e42626f8'
COLLECTION_ID = '66cccc04ad19ca34f89b5f08'


#tell flask to read home page
@app.route('/')
def index(): 
    return render_template('apphome.html')

#tell flask to read about page
#@app.route('/about')
#def about(): 
    #return render_template('about.html')

#tell flask to read contact page
#@app.route('/contact')
#def conact(): 
    #return render_template('contact.html')

#tell flask to read defs page
@app.route('/defs')
def defs(): 
    return render_template('defs.html')

#tell flask to read dashboard page
@app.route('/dashboard')
def dashboard():
    return render_template('dashboard.html')

#tell flask to read charttest
#@app.route('/charttest')
#def charttest():
#    return render_template('charttest.html')

#Tell Flask to create a record
@app.route('/add-feature', methods=['POST'])
def add_feature():
    new_data = request.json  # Get the new feature from the request body
    # Fetch the current data from JSONBin
    headers = {
        'X-Master-Key': API_KEY,
        'X-Collection-Id': COLLECTION_ID
    }
    url = f'https://api.jsonbin.io/v3/b/{BIN_ID}/latest'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        bin_data = response.json()
        # Add the new feature to the features array
        bin_data['record']['features'].append(new_data)
        # Update the bin with the new data
        update_url = f'https://api.jsonbin.io/v3/b/{BIN_ID}'
        update_response = requests.put(update_url, headers=headers, json=bin_data)

     #   if update_response.status_code == 200:
     #       return jsonify({"message": "Data added successfully"}), 200
      #  else:
      #      return jsonify({"error": "Failed to update data"}), 500
    #else:
     #   return jsonify({"error": "Failed to retrieve current data"}), 500
#Tell Flask how to read bin - map
@app.route('/get-map-data',  methods=['GET'])
def get_map_data():
    headers = {
        'X-Master-Key': JSONBIN_ACCESS_KEY,
    }
    response = requests.get(JSONBIN_API_URL, headers=headers)

    if response.status_code == 200:
        return jsonify(response.json()), 200
    else:
        return jsonify({'error': 'Failed to fetch data from JSONBin.io'}), response.status_code





#tell flask run app
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Sep/2024 10:34:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2024 10:34:11] "GET /static/bootstrap/css/bootstrap-theme.min.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Sep/2024 10:34:11] "GET /static/bootstrap-slider/css/bootstrap-slider.min.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Sep/2024 10:34:12] "GET /static/nearmiss1.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Sep/2024 10:34:12] "GET /static/stylesheet.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Sep/2024 10:34:12] "GET /static/PlanRVA_White.png HTTP/1.1" 304 -
[2024-09-03 10:34:39,149] ERROR in app: Exception on /add-feature [POST]
Traceback (most recent call last):
  File "c:\Users\egreenwell\anaconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\egreenwell\anaconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user